In [342]:
import os
import pandas as pd
import csv
import re
import numpy as np
import datetime
import pytz
from google.cloud import bigquery
from google.cloud.bigquery import SchemaField


## Process Roster CSV file

In [1082]:

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Roster/')

file="meeting-26886-results.csv"


df = pd.read_csv(file, dtype=str)


In [1083]:
df.describe()

,MeetingId,MeetingName,MeetingDate,VenueName,Organiser,City,Country,Season,EntryId,StartListId,...,Round24Attempts,Round24TargetHeight,Round24Wind,Round25Status,Round25Record,Round25Records,Round25Result,Round25Attempts,Round25TargetHeight,Round25Wind
count,4320,4320,4320,4320,4320,4320,4320,4320,4320,4320,...,1,6,0,6,0,0,0,1,6,0
unique,1,1,1,1,1,1,1,1,4320,217,...,1,1,0,1,0,0,0,1,1,0
top,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451719,313732,...,xo,4.75,NaN,ok,NaN,NaN,NaN,xxx,4.85,NaN
freq,4320,4320,4320,4320,4320,4320,4320,4320,1,124,...,1,6,NaN,6,NaN,NaN,NaN,1,6,NaN


In [1084]:
#df.info(verbose=True, show_counts=True)

In [1085]:
df

,MeetingId,MeetingName,MeetingDate,VenueName,Organiser,City,Country,Season,EntryId,StartListId,...,Round24Attempts,Round24TargetHeight,Round24Wind,Round25Status,Round25Record,Round25Records,Round25Result,Round25Attempts,Round25TargetHeight,Round25Wind
0,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451719,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451722,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451714,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451705,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451717,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2452695,313732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4316,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2452696,313732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4317,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2452648,313732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4318,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2452649,313732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1086]:
df.columns

Index(['MeetingId', 'MeetingName', 'MeetingDate', 'VenueName', 'Organiser',
       'City', 'Country', 'Season', 'EntryId', 'StartListId',
       ...
       'Round24Attempts', 'Round24TargetHeight', 'Round24Wind',
       'Round25Status', 'Round25Record', 'Round25Records', 'Round25Result',
       'Round25Attempts', 'Round25TargetHeight', 'Round25Wind'],
      dtype='object', length=247)

In [1087]:
'''
# Select separate columns


df_filtered = df[['MeetingName', 'VenueName', 'City', 'EntryId', 'RelayTeamName', 'FullName', 'FirstName','MiddleName', 'LastName','EventGender', 'ParaClassRunJump',
                 'ParaClassThrow', 'NotesPublic', 'CountryCode', 'DateOfBirth', 'EventStart', 'EventName','PrimaryImplementValue', 'PrimaryImplementUnit', 'EventStage', 'ClubName', 'BibNumber', 
                  'StartStatus', 'PersonalBest', 'SeasonBest', 'Lane', 'Place', 'AgeGroup', 'Title', 'WindReading', 'Promotion', 'HeatName', 'Result', 'ReactionTime','SeedingResult', 'ResultNET', 
                  'SplitTime', 'LapTime', 'RelayId', 'NotesPublic', 'ResultRecords']].copy()

df_attempts = df.iloc[:, 72:]

'''

"\n# Select separate columns\n\n\ndf_filtered = df[['MeetingName', 'VenueName', 'City', 'EntryId', 'RelayTeamName', 'FullName', 'FirstName','MiddleName', 'LastName','EventGender', 'ParaClassRunJump',\n                 'ParaClassThrow', 'NotesPublic', 'CountryCode', 'DateOfBirth', 'EventStart', 'EventName','PrimaryImplementValue', 'PrimaryImplementUnit', 'EventStage', 'ClubName', 'BibNumber', \n                  'StartStatus', 'PersonalBest', 'SeasonBest', 'Lane', 'Place', 'AgeGroup', 'Title', 'WindReading', 'Promotion', 'HeatName', 'Result', 'ReactionTime','SeedingResult', 'ResultNET', \n                  'SplitTime', 'LapTime', 'RelayId', 'NotesPublic', 'ResultRecords']].copy()\n\ndf_attempts = df.iloc[:, 72:]\n\n"

In [1088]:
'''
NotesPublic - REMARKS
RelayId - Do Not Map
LapTime - LAP_TIME
SplitTime - SPLIT_TIME
(PersonalBest - REMARKS2)   remove
(SeasonBest - REMARKS3)   remove
ResultNET - NET_RESULT
SeedingResult - SEED
Promotion - QUALIFICATION
ReactionTime - RX_TIME
Result - RESULT
HeatName - HEAT
WindReading - WIND
AgeGroup - GROUP
Title - SUB_EVENT1
Place - RANK
Lane - LANE
StartStatus - DNF/DNS/DQ/NM to RESULT
BibNumber - TAG_ID
ClubName - TEAM
EventStage - STAGE
PrimaryImplementValue - EVENT_CLASS1
PrimaryImplementUnit - EVENT_CLASS2
EventName - EVENT
EventStart - DATE
DateOfBirth - DOB
CountryCode - NATIONALITY
ParaClassThrow - SUB_EVENT2
ParaClassRunJump - SUB_EVENT3
EventGender - GENDER
LastName - LAST_NAME
MiddleName - OTHER_NAME
FirstName - FIRST_NAME
FullName - NAME1
RelayTeamName - NAME2 (if relay event only)
EntryId - ATHLETE_ID
City - CITY
VenueName - VENUE
MeetingName - COMPETITION
ResultRecords - SB/PB(maybe map into REMARKS)

EVENT_CLASS = PrimaryImplementValue(EVENT_CLASS1) + ' ' + PrimaryImplementUnit(EVENT_CLASS2)
REMARKS = Title + ' ' + ParaClassThrow + ' ' + ParaClassRunJump (if paraclassthrow or paraclassrunjump are not null then SUB_EVENT=Para

Need to check if SPLIT_TIME, LAP_TIME and NET_RESULT are used
'''



athletes_selected = df.rename(columns={
'NotesPublic': 'REMARKS', 'LapTime': "LAP_TIME", 'SplitTime': "SPLIT_TIME", 'ResultNET': "NET_TIME", 'SeedingResult': "SEED", 
 'Promotion': "QUALIFICATION", 'ReactionTime': "RX_TIME", 'Result': "RESULT", 'HeatName': "HEAT", 'WindReading': "WIND", 
 'AgeGroup': "GROUP", 'Title': "SUB_EVENT1", 'Place': "RANK", 'Lane': "LANE", 'Place': "RANK", 'BibNumber': "TAG_ID", 'ClubName': "TEAM",
 'ClubName': "TEAM", 'EventStage': "STAGE", 'PrimaryImplementValue': "EVENT_CLASS1", 'PrimaryImplementUnit': "EVENT_CLASS2",
 'EventName': "EVENT", 'EventStart': "DATE", 'DateOfBirth': "DOB", 'CountryCode': "NATIONALITY", 'ParaClassThrow': "SUB_EVENT2", 
 'ParaClassThrow': "SUB_EVENT2", 'ParaClassRunJump': "SUB_EVENT3", 'EventGender': "GENDER", 'LastName': "LAST_NAME", 'MiddleName': "OTHER_NAME", 
 'FirstName': "FIRST_NAME", 'FullName': "NAME1", 'RelayTeamName': "NAME2", 'EntryId': "ATHLETE_ID", 'City': "HOST_CITY",  'VenueName': "VENUE",
 'MeetingName': "COMPETITION"
})

In [1089]:
athletes_selected

,MeetingId,COMPETITION,MeetingDate,VENUE,Organiser,HOST_CITY,Country,Season,ATHLETE_ID,StartListId,...,Round24Attempts,Round24TargetHeight,Round24Wind,Round25Status,Round25Record,Round25Records,Round25Result,Round25Attempts,Round25TargetHeight,Round25Wind
0,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451719,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451722,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451714,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451705,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451717,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2452695,313732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4316,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2452696,313732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4317,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2452648,313732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4318,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2452649,313732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1090]:
#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Roster/')

#df_attempts.to_csv('attempts.csv', encoding='utf-8')


In [1091]:
'''
# Process track events
# Build a list of dicts for each original row
# Does not include HJ and PV

round_numbers = sorted({
    int(re.search(r'Round(\d+)Status', col).group(1))
    for col in df_filtered.columns if re.match(r'Round\d+Status', col)
})


valid_rounds_list = []
for _, row in df_filtered.iterrows():
    rounds_dict = {}
    for rnd in round_numbers:
        status_col = f'Round{rnd}Status'
        result_col = f'Round{rnd}Result'
        wind_col = f'Round{rnd}Wind'

        result_val = row[result_col]

        # Only include if status is 'ok' AND result is not NaN
        if (
            str(row[status_col]).strip().lower() == 'ok'
            and pd.notna(result_val)
        ):
            rounds_dict[rnd] = {
                'Result': result_val,
                'Wind': row[wind_col]
            }
    valid_rounds_list.append(rounds_dict)

# Create new DataFrame
track_df = pd.DataFrame({
    'DICT_RESULTS': valid_rounds_list
})
'''

"\n# Process track events\n# Build a list of dicts for each original row\n# Does not include HJ and PV\n\nround_numbers = sorted({\n    int(re.search(r'Round(\\d+)Status', col).group(1))\n    for col in df_filtered.columns if re.match(r'Round\\d+Status', col)\n})\n\n\nvalid_rounds_list = []\nfor _, row in df_filtered.iterrows():\n    rounds_dict = {}\n    for rnd in round_numbers:\n        status_col = f'Round{rnd}Status'\n        result_col = f'Round{rnd}Result'\n        wind_col = f'Round{rnd}Wind'\n\n        result_val = row[result_col]\n\n        # Only include if status is 'ok' AND result is not NaN\n        if (\n            str(row[status_col]).strip().lower() == 'ok'\n            and pd.notna(result_val)\n        ):\n            rounds_dict[rnd] = {\n                'Result': result_val,\n                'Wind': row[wind_col]\n            }\n    valid_rounds_list.append(rounds_dict)\n\n# Create new DataFrame\ntrack_df = pd.DataFrame({\n    'DICT_RESULTS': valid_rounds_list\n})\

In [1092]:
'''

# Process field events

round_numbers = sorted({
    int(re.search(r'Round(\d+)Status', col).group(1))
    for col in df_attempts.columns if re.match(r'Round\d+Status', col)
})

target_height_list = []
for _, row in df_attempts.iterrows():
    height_dict = {}
    for rnd in round_numbers:
        status_col = f'Round{rnd}Status'
        target_col = f'Round{rnd}TargetHeight'
        attempts_col = f'Round{rnd}Attempts'

        if str(row[status_col]).strip().lower() == 'ok' and pd.notna(row[target_col]) and pd.notna(row[attempts_col]):
            height_dict[row[target_col]] = row[attempts_col]
    target_height_list.append(height_dict)

# Create DataFrame
temp_df = pd.DataFrame({'DICT_RESULTS': target_height_list})
'''

"\n\n# Process field events\n\nround_numbers = sorted({\n    int(re.search(r'Round(\\d+)Status', col).group(1))\n    for col in df_attempts.columns if re.match(r'Round\\d+Status', col)\n})\n\ntarget_height_list = []\nfor _, row in df_attempts.iterrows():\n    height_dict = {}\n    for rnd in round_numbers:\n        status_col = f'Round{rnd}Status'\n        target_col = f'Round{rnd}TargetHeight'\n        attempts_col = f'Round{rnd}Attempts'\n\n        if str(row[status_col]).strip().lower() == 'ok' and pd.notna(row[target_col]) and pd.notna(row[attempts_col]):\n            height_dict[row[target_col]] = row[attempts_col]\n    target_height_list.append(height_dict)\n\n# Create DataFrame\ntemp_df = pd.DataFrame({'DICT_RESULTS': target_height_list})\n"

In [1093]:
'''
# Consolidated code
# VERSION 1

# Find all round numbers present
round_numbers = sorted({
    int(re.search(r'Round(\d+)Status', col).group(1))
    for col in df.columns if re.match(r'Round\d+Status', col)
})

dict_results_list = []

for _, row in df.iterrows():
    rounds_dict = {}

    # CASE 1: High Jump & Pole Vault (field events)
    if row.get("EventName", "").strip().lower() in ["high jump", "pole vault"]:
        for rnd in round_numbers:
            status_col = f'Round{rnd}Status'
            result_col = f'Round{rnd}Result'
            attempts_col = f'Round{rnd}Attempts'

            if (
                str(row.get(status_col, "")).strip().lower() == "ok"
                and pd.notna(row.get(result_col))
                and pd.notna(row.get(attempts_col))
            ):
                rounds_dict[row[result_col]] = row[attempts_col]

    # CASE 2: All other events → Result/Wind
    else:
        for rnd in round_numbers:
            status_col = f'Round{rnd}Status'
            result_col = f'Round{rnd}Result'
            wind_col = f'Round{rnd}Wind'

            if (
                str(row.get(status_col, "")).strip().lower() == "ok"
                and pd.notna(row.get(result_col))
            ):
                rounds_dict[rnd] = {
                    "Result": row[result_col],
                    "Wind": row.get(wind_col)
                }

    dict_results_list.append(rounds_dict)

df["DICT_RESULTS"] = dict_results_list
'''

'\n# Consolidated code\n# VERSION 1\n\n# Find all round numbers present\nround_numbers = sorted({\n    int(re.search(r\'Round(\\d+)Status\', col).group(1))\n    for col in df.columns if re.match(r\'Round\\d+Status\', col)\n})\n\ndict_results_list = []\n\nfor _, row in df.iterrows():\n    rounds_dict = {}\n\n    # CASE 1: High Jump & Pole Vault (field events)\n    if row.get("EventName", "").strip().lower() in ["high jump", "pole vault"]:\n        for rnd in round_numbers:\n            status_col = f\'Round{rnd}Status\'\n            result_col = f\'Round{rnd}Result\'\n            attempts_col = f\'Round{rnd}Attempts\'\n\n            if (\n                str(row.get(status_col, "")).strip().lower() == "ok"\n                and pd.notna(row.get(result_col))\n                and pd.notna(row.get(attempts_col))\n            ):\n                rounds_dict[row[result_col]] = row[attempts_col]\n\n    # CASE 2: All other events → Result/Wind\n    else:\n        for rnd in round_numbers:\n    

In [1094]:
'''
# VERSION 2
# Get all round numbers present
round_numbers = sorted({
    int(re.search(r'Round(\d+)Status', col).group(1))
    for col in df.columns if re.match(r'Round\d+Status', col)
})

dict_results_list = []

for _, row in df.iterrows():
    rounds_dict = {}

    # CASE 1: High Jump & Pole Vault
    if str(row.get("EventName", "")).strip().lower() in ["high jump", "pole vault"]:
        for rnd in round_numbers:
            status_col = f'Round{rnd}Status'
            result_col = f'Round{rnd}Result'
            attempts_col = f'Round{rnd}Attempts'

            if (
                str(row.get(status_col, "")).strip().lower() == "ok"
                and pd.notna(row.get(result_col))
                and pd.notna(row.get(attempts_col))  # allows 'xxx' etc.
            ):
                rounds_dict[row[result_col]] = row[attempts_col]

    # CASE 2: All other events → Result/Wind
    else:
        for rnd in round_numbers:
            status_col = f'Round{rnd}Status'
            result_col = f'Round{rnd}Result'
            wind_col = f'Round{rnd}Wind'

            if (
                str(row.get(status_col, "")).strip().lower() == "ok"
                and pd.notna(row.get(result_col))
            ):
                rounds_dict[rnd] = {
                    "Result": row[result_col],
                    "Wind": row.get(wind_col)
                }

    dict_results_list.append(rounds_dict)

# Add to DataFrame
df["DICT_RESULTS"] = dict_results_list
'''

'\n# VERSION 2\n# Get all round numbers present\nround_numbers = sorted({\n    int(re.search(r\'Round(\\d+)Status\', col).group(1))\n    for col in df.columns if re.match(r\'Round\\d+Status\', col)\n})\n\ndict_results_list = []\n\nfor _, row in df.iterrows():\n    rounds_dict = {}\n\n    # CASE 1: High Jump & Pole Vault\n    if str(row.get("EventName", "")).strip().lower() in ["high jump", "pole vault"]:\n        for rnd in round_numbers:\n            status_col = f\'Round{rnd}Status\'\n            result_col = f\'Round{rnd}Result\'\n            attempts_col = f\'Round{rnd}Attempts\'\n\n            if (\n                str(row.get(status_col, "")).strip().lower() == "ok"\n                and pd.notna(row.get(result_col))\n                and pd.notna(row.get(attempts_col))  # allows \'xxx\' etc.\n            ):\n                rounds_dict[row[result_col]] = row[attempts_col]\n\n    # CASE 2: All other events → Result/Wind\n    else:\n        for rnd in round_numbers:\n            sta

In [1095]:
# VERSION 3
# detect round numbers
round_numbers = sorted({
    int(re.search(r'Round(\d+)Status', col).group(1))
    for col in df.columns if re.match(r'Round\d+Status', col)
})

def build_dict_with_fallback(row):
    d = {}
    eventname = str(row.get("EVENT", "")).strip().lower()

    # FIELD EVENTS: only high jump & pole vault (store result->attempts)
    if eventname in ["high jump", "pole vault"]:
        for rnd in round_numbers:
            status_col = f"Round{rnd}Status"
            result_col = f"Round{rnd}Result"
            attempts_col = f"Round{rnd}Attempts"
            target_col = f"Round{rnd}TargetHeight"

            status = str(row.get(status_col, "")).strip().lower()
            # include attempts for status == ok and attempts not missing
            if status == "ok" and pd.notna(row.get(attempts_col)):
                # choose key: prefer Result, then TargetHeight, else RoundN
                if pd.notna(row.get(result_col)):
                    key = row[result_col]
                elif pd.notna(row.get(target_col)):
                    key = row[target_col]
                else:
                    key = f"Round{rnd}"
                d[key] = row.get(attempts_col)

    # OTHER EVENTS: keep round_number -> {Result, Wind}
    else:
        for rnd in round_numbers:
            status_col = f"Round{rnd}Status"
            result_col = f"Round{rnd}Result"
            wind_col = f"Round{rnd}Wind"

            status = str(row.get(status_col, "")).strip().lower()
            if status == "ok" and pd.notna(row.get(result_col)):
                d[rnd] = {"RESULT": row[result_col], "WIND": row.get(wind_col)}

    return d

# apply and store
athletes_selected["DICT_RESULTS"] = athletes_selected.apply(build_dict_with_fallback, axis=1)

In [1096]:
'''
1️⃣ Load the file and keep all values as strings
python
Copy
Edit
df = pd.read_csv("/mnt/data/meeting-26886-results.csv", dtype=str).replace({None: pd.NA})
dtype=str makes sure even numbers or "xxx" stay as text (important because "xxx" would be NaN if read as numeric).

.replace({None: pd.NA}) standardizes missing values to Pandas’ NA type.

2️⃣ Detect which rounds exist in the file
python
Copy
Edit
round_numbers = sorted({
    int(re.search(r'Round(\d+)Status', col).group(1))
    for col in df.columns if re.match(r'Round\d+Status', col)
})
This scans all column names.

Picks out those matching Round1Status, Round2Status, etc.

Extracts the number (1, 2, 3, …) and makes a sorted list.
Example: [1, 2, 3, 4, 5, 6].

3️⃣ Define a function to build the dictionary for each row
python
Copy
Edit
def build_dict_with_fallback(row):
    d = {}
    eventname = str(row.get("EventName", "")).strip().lower()
Each row in the DataFrame gets its own dictionary.

We look at the event name so we know which branch of logic to apply.

4️⃣ Field events logic (High Jump / Pole Vault)
python
Copy
Edit
if eventname in ["high jump", "pole vault"]:
    for rnd in round_numbers:
        status_col = f"Round{rnd}Status"
        result_col = f"Round{rnd}Result"
        attempts_col = f"Round{rnd}Attempts"
        target_col = f"Round{rnd}TargetHeight"

        status = str(row.get(status_col, "")).strip().lower()
        if status == "ok" and pd.notna(row.get(attempts_col)):
            if pd.notna(row.get(result_col)):
                key = row[result_col]
            elif pd.notna(row.get(target_col)):
                key = row[target_col]
            else:
                key = f"Round{rnd}"
            d[key] = row.get(attempts_col)
Only process if event name is High Jump or Pole Vault.

For each round:

Check RoundNStatus is "ok".

Check Attempts is not missing (this is where "xxx" still counts as valid).

Decide the dictionary key:

Prefer RoundResult (e.g., "1.80").

If missing, use TargetHeight (e.g., "1.75").

If both missing, use "RoundN" as a fallback.

Value is always Attempts (e.g., "xo", "xxx").

5️⃣ Other events logic (Results + Wind)
python
Copy
Edit
else:
    for rnd in round_numbers:
        status_col = f"Round{rnd}Status"
        result_col = f"Round{rnd}Result"
        wind_col = f"Round{rnd}Wind"

        status = str(row.get(status_col, "")).strip().lower()
        if status == "ok" and pd.notna(row.get(result_col)):
            d[rnd] = {"Result": row[result_col], "Wind": row.get(wind_col)}
All non-high-jump / non-pole-vault events go here.

Key is the round number (e.g., 1, 2, 3).

Value is another dictionary with Result and Wind.

6️⃣ Apply the function to the DataFrame
python
Copy
Edit
df["DICT_RESULTS"] = df.apply(build_dict_with_fallback, axis=1)
Runs the logic for each row.

Creates a single column where each row’s value is the dictionary we built.

7️⃣ Quick check for "xxx" inclusion
python
Copy
Edit
def has_xxx(d):
    return any(isinstance(v, str) and "xxx" in v.lower() for v in d.values()) or \
           any(isinstance(k, str) and "xxx" in k.lower() for k in d.keys())
Looks for "xxx" in either keys or values of the dictionary.

Ensures those failed attempts are actually present.

'''

'\n1️⃣ Load the file and keep all values as strings\npython\nCopy\nEdit\ndf = pd.read_csv("/mnt/data/meeting-26886-results.csv", dtype=str).replace({None: pd.NA})\ndtype=str makes sure even numbers or "xxx" stay as text (important because "xxx" would be NaN if read as numeric).\n\n.replace({None: pd.NA}) standardizes missing values to Pandas’ NA type.\n\n2️⃣ Detect which rounds exist in the file\npython\nCopy\nEdit\nround_numbers = sorted({\n    int(re.search(r\'Round(\\d+)Status\', col).group(1))\n    for col in df.columns if re.match(r\'Round\\d+Status\', col)\n})\nThis scans all column names.\n\nPicks out those matching Round1Status, Round2Status, etc.\n\nExtracts the number (1, 2, 3, …) and makes a sorted list.\nExample: [1, 2, 3, 4, 5, 6].\n\n3️⃣ Define a function to build the dictionary for each row\npython\nCopy\nEdit\ndef build_dict_with_fallback(row):\n    d = {}\n    eventname = str(row.get("EventName", "")).strip().lower()\nEach row in the DataFrame gets its own dictionary.\

In [1097]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Roster/')

athletes_selected.to_csv('check.csv', encoding='utf-8')


In [1098]:
# Potential new fields to map across:

# tilastopajaid
# relayid
# paraclass - EVENT_CLASS
# split time
# lap time
# resultnet



In [1099]:
# Put relay names into a list - must map RelayTeamName(NAME2) to FullName(NAME1) for relay events at the end
# Forward-fill the team name so each member gets the right one
athletes_selected["NAME2"] = athletes_selected["NAME2"].replace("", pd.NA).ffill()

# Remove the pure team name row (where FullName is NaN or empty)
members_df = athletes_selected[athletes_selected["NAME1"].notna() & (athletes_selected["NAME1"] != "")]

# Group by RelayId
relay_teams = (
    members_df.groupby("RelayId")
    .apply(lambda g: pd.Series({
        "TEAM_NAME": g["NAME2"].iloc[0],
        "MEMBERS": list(g["NAME1"])
    }))
    .reset_index(drop=True)
)

# Create mapping from RelayTeamName -> Members
relay_mapping = relay_teams.set_index("TEAM_NAME")["MEMBERS"].to_dict()

# Suppose df_dest is your destination DataFrame
# This will insert the Members list into NAME where RelayTeamName matches
#athletes_selected["NAME1"] = athletes_selected["NAME2"].map(relay_mapping).combine_first(athletes_selected["NAME1"])

mask = athletes_selected["EVENT"].isin(["4x400m", "4x100m", "4x100m MIXED TEAM", "4x400m MIXED TEAM", "2x50m MIXED TEAM"])
athletes_selected.loc[mask, "NAME1"] = (athletes_selected.loc[mask, "NAME2"].map(relay_mapping).combine_first(athletes_selected.loc[mask, "NAME1"])
)

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_23004/742416499.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [1100]:
relay_teams

,TEAM_NAME,MEMBERS
0,OMG. The weather is sooooo hot,"[Kian Boon Lim, Jiayi Lim]"
1,SeeeK Family,"[Kenny Wai Seng Leong, Elissa En Hui Leong]"
2,Lucas Cheng Team,"[Tze Wee John Cheng, Sheng Tze Lucas Cheng]"
3,Zeldy,"[Felix Choo, Zelda Choo]"
4,McNuggets Eating Club,"[Harry Loh, Rachel Loh]"
...,...,...
227,Aden Seow & Parent,"[Jason Au, Aden Kai qing Seow]"
228,Jaden Seow & Parent,"[Jason Au, Jaden Yuan Qing Seow]"
229,Hannah Wong & Parent,"[Charlene Mingyan Han, Hannah Danielle Wong]"
230,Black,"[Javier Loh, Aden Tan, Ye Tao Tong, Siang Yang..."


In [1101]:
'''
Step 1: members_df.groupby("RelayId")
This tells pandas:

“Split my DataFrame into chunks (groups) where all rows in a chunk have the same RelayId.”

Example — from your CSV:

nginx
Copy
Edit
RelayId   FullName         RelayTeamName
2451916   Ching Lee        Team Cicada Trackers
2451916   Zurie Chong      Team Cicada Trackers
2451916   Kristy Kau       Team Cicada Trackers
2451916   Riana Kaul       Team Cicada Trackers
→ All these rows become one group because they have the same RelayId.

Step 2: .apply(lambda g: ...)
The .apply() method says:

“For each group g created by groupby, run this custom function and return the results.”

Here, g is a mini DataFrame containing just the rows for one RelayId.

Step 3: pd.Series({ ... }) inside apply
Inside the function, we:

Take the first team name from the group:
g["RelayTeamName"].iloc[0]
(Since all rows in a group have the same team name, we just grab the first.)

Turn all the names in FullName into a Python list:
list(g["FullName"])

We pack these into a pd.Series:

python
Copy
Edit
pd.Series({
    "RelayTeamName": g["RelayTeamName"].iloc[0],
    "Members": list(g["FullName"])
})
This is the summary row for that team.

Step 4: .reset_index(drop=True)
groupby.apply() returns a DataFrame where the group keys (RelayId) are part of the index.
We call .reset_index(drop=True) to:

Throw away the old index

Give the result a fresh 0..N index

✅ End result:
A clean DataFrame with one row per relay team, and:

RelayTeamName → team’s name

Members → list of names

'''

'\nStep 1: members_df.groupby("RelayId")\nThis tells pandas:\n\n“Split my DataFrame into chunks (groups) where all rows in a chunk have the same RelayId.”\n\nExample — from your CSV:\n\nnginx\nCopy\nEdit\nRelayId   FullName         RelayTeamName\n2451916   Ching Lee        Team Cicada Trackers\n2451916   Zurie Chong      Team Cicada Trackers\n2451916   Kristy Kau       Team Cicada Trackers\n2451916   Riana Kaul       Team Cicada Trackers\n→ All these rows become one group because they have the same RelayId.\n\nStep 2: .apply(lambda g: ...)\nThe .apply() method says:\n\n“For each group g created by groupby, run this custom function and return the results.”\n\nHere, g is a mini DataFrame containing just the rows for one RelayId.\n\nStep 3: pd.Series({ ... }) inside apply\nInside the function, we:\n\nTake the first team name from the group:\ng["RelayTeamName"].iloc[0]\n(Since all rows in a group have the same team name, we just grab the first.)\n\nTurn all the names in FullName into a Pyt

In [1102]:
'''
1. Original Data
pgsql
Copy
Edit
RelayTeamName         FullName        RelayId
---------------------------------------------------
Team Cicada Trackers  Ching Lee       2451916
NaN                   Zurie Chong     2451916
NaN                   Kristy Kau      2451916
NaN                   Riana Kaul      2451916
Team Road Runners     John Tan        2487215
NaN                   Alex Wong       2487215
NaN                   Sarah Lim       2487215
2. Step 1: groupby("RelayId")
We now have two groups:

Group 1 — RelayId 2451916

pgsql
Copy
Edit
RelayTeamName         FullName        RelayId
---------------------------------------------------
Team Cicada Trackers  Ching Lee       2451916
NaN                   Zurie Chong     2451916
NaN                   Kristy Kau      2451916
NaN                   Riana Kaul      2451916
Group 2 — RelayId 2487215

markdown
Copy
Edit
RelayTeamName         FullName        RelayId
---------------------------------------------------
Team Road Runners     John Tan        2487215
NaN                   Alex Wong       2487215
NaN                   Sarah Lim       2487215
3. Step 2: .apply(lambda g: ...)
We now process each mini-DataFrame (g) separately.

For Group 1:

python
Copy
Edit
RelayTeamName = g["RelayTeamName"].iloc[0]   # "Team Cicada Trackers"
Members = list(g["FullName"])                # ["Ching Lee", "Zurie Chong", "Kristy Kau", "Riana Kaul"]
For Group 2:

python
Copy
Edit
RelayTeamName = g["RelayTeamName"].iloc[0]   # "Team Road Runners"
Members = list(g["FullName"])                # ["John Tan", "Alex Wong", "Sarah Lim"]
4. Step 3: Combine Results
After processing each group, pandas stacks the results into a new DataFrame:

markdown
Copy
Edit
RelayTeamName         Members
---------------------------------------------------
Team Cicada Trackers  [Ching Lee, Zurie Chong, Kristy Kau, Riana Kaul]
Team Road Runners     [John Tan, Alex Wong, Sarah Lim]
💡 Key point:
groupby → splits data into chunks → apply runs custom aggregation per chunk → results are recombined into one DataFrame.

'''

'\n1. Original Data\npgsql\nCopy\nEdit\nRelayTeamName         FullName        RelayId\n---------------------------------------------------\nTeam Cicada Trackers  Ching Lee       2451916\nNaN                   Zurie Chong     2451916\nNaN                   Kristy Kau      2451916\nNaN                   Riana Kaul      2451916\nTeam Road Runners     John Tan        2487215\nNaN                   Alex Wong       2487215\nNaN                   Sarah Lim       2487215\n2. Step 1: groupby("RelayId")\nWe now have two groups:\n\nGroup 1 — RelayId 2451916\n\npgsql\nCopy\nEdit\nRelayTeamName         FullName        RelayId\n---------------------------------------------------\nTeam Cicada Trackers  Ching Lee       2451916\nNaN                   Zurie Chong     2451916\nNaN                   Kristy Kau      2451916\nNaN                   Riana Kaul      2451916\nGroup 2 — RelayId 2487215\n\nmarkdown\nCopy\nEdit\nRelayTeamName         FullName        RelayId\n---------------------------------------

In [1103]:
athletes_selected

,MeetingId,COMPETITION,MeetingDate,VENUE,Organiser,HOST_CITY,Country,Season,ATHLETE_ID,StartListId,...,Round24TargetHeight,Round24Wind,Round25Status,Round25Record,Round25Records,Round25Result,Round25Attempts,Round25TargetHeight,Round25Wind,DICT_RESULTS
0,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451719,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{1: {'RESULT': '13.656', 'WIND': '-1.4'}}"
1,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451722,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{1: {'RESULT': '13.716', 'WIND': '-1.4'}}"
2,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451714,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{1: {'RESULT': '13.960', 'WIND': '-1.3'}}"
3,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451705,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{1: {'RESULT': '14.119', 'WIND': '-1.8'}}"
4,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2451717,313711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{1: {'RESULT': '14.132', 'WIND': '-1.3'}}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2452695,313732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{}
4316,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2452696,313732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{}
4317,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2452648,313732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{}
4318,26886,Pesta Sukan Athletics 2025,2025-07-26 08:30:00,Home of Athletics,Singapore Athletics,Singapore,Singapore,Outdoor,2452649,313732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{}


In [1104]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Roster/')


athletes_selected.to_csv('check.csv', sep=',', encoding='utf-8-sig', index=False)

In [1105]:
athletes_selected.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4320 entries, 0 to 4319
Data columns (total 248 columns):
 #    Column                   Non-Null Count  Dtype 
---   ------                   --------------  ----- 
 0    MeetingId                4320 non-null   object
 1    COMPETITION              4320 non-null   object
 2    MeetingDate              4320 non-null   object
 3    VENUE                    4320 non-null   object
 4    Organiser                4320 non-null   object
 5    HOST_CITY                4320 non-null   object
 6    Country                  4320 non-null   object
 7    Season                   4320 non-null   object
 8    ATHLETE_ID               4320 non-null   object
 9    StartListId              4320 non-null   object
 10   SUB_EVENT1               814 non-null    object
 11   NAME2                    1720 non-null   object
 12   NationalRelayTeam        238 non-null    object
 13   NAME1                    4315 non-null   object
 14   FIRST_NAME            

In [1106]:
athletes_selected['DICT_RESULTS']

0       {1: {'RESULT': '13.656', 'WIND': '-1.4'}}
1       {1: {'RESULT': '13.716', 'WIND': '-1.4'}}
2       {1: {'RESULT': '13.960', 'WIND': '-1.3'}}
3       {1: {'RESULT': '14.119', 'WIND': '-1.8'}}
4       {1: {'RESULT': '14.132', 'WIND': '-1.3'}}
                          ...                    
4315                                           {}
4316                                           {}
4317                                           {}
4318                                           {}
4319                                           {}
Name: DICT_RESULTS, Length: 4320, dtype: object

In [1107]:
# Drop attempts columns

df_final = athletes_selected.loc[:, ~athletes_selected.columns.str.contains(r'Round', case=False, na=False)]


In [1108]:
df_final.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4320 entries, 0 to 4319
Data columns (total 72 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   MeetingId                4320 non-null   object
 1   COMPETITION              4320 non-null   object
 2   MeetingDate              4320 non-null   object
 3   VENUE                    4320 non-null   object
 4   Organiser                4320 non-null   object
 5   HOST_CITY                4320 non-null   object
 6   Country                  4320 non-null   object
 7   Season                   4320 non-null   object
 8   ATHLETE_ID               4320 non-null   object
 9   StartListId              4320 non-null   object
 10  SUB_EVENT1               814 non-null    object
 11  NAME2                    1720 non-null   object
 12  NationalRelayTeam        238 non-null    object
 13  NAME1                    4315 non-null   object
 14  FIRST_NAME               4082 non-null  

In [1109]:
'''
NotesPublic - REMARKS1
RelayId - Do Not Map
LapTime - LAP_TIME
SplitTime - SPLIT_TIME
(PersonalBest - REMARKS2)   remove
(SeasonBest - REMARKS3)   remove
ResultNET - NET_RESULT
SeedingResult - SEED
Promotion - QUALIFICATION
ReactionTime - RX_TIME
Result - RESULT
HeatName - HEAT
WindReading - WIND
AgeGroup - GROUP
Title - SUB_EVENT1
Place - RANK
Lane - LANE
StartStatus - DNF/DNS/DQ/NM map to RESULT
BibNumber - TAG_ID
ClubName - TEAM
EventStage - STAGE
PrimaryImplementValue - EVENT_CLASS1
PrimaryImplementUnit - EVENT_CLASS2
EventName - EVENT
EventStart - DATE
DateOfBirth - DOB
CountryCode - NATIONALITY
ParaClassThrow - SUB_EVENT2
ParaClassRunJump - SUB_EVENT3
EventGender - GENDER
LastName - LAST_NAME
MiddleName - OTHER_NAME
FirstName - FIRST_NAME
FullName - NAME1
RelayTeamName - NAME2 (if relay event only)
EntryId - ATHLETE_ID
City - CITY
VenueName - VENUE
MeetingName - COMPETITION
ResultRecords - SB/PB(maybe map into REMARKS)

EVENT_CLASS = PrimaryImplementValue(EVENT_CLASS1) + ' ' + PrimaryImplementUnit(EVENT_CLASS2)
REMARKS = Title + ' ' + ParaClassThrow + ' ' + ParaClassRunJump (if paraclassthrow or paraclassrunjump are not null then SUB_EVENT=Para

Need to check if SPLIT_TIME, LAP_TIME and NET_RESULT are used
'''

df_final['EVENT_CLASS'] = df_final['EVENT_CLASS1'] + df_final['EVENT_CLASS2']
#df_final['REMARKS'] = df_final['SUB_EVENT1'].astype(str) + ' ' + df_final['REMARKS'].astype(str) + ' ' + df_final['ResultRecords'].astype(str) + ' ' + df_final['SUB_EVENT2'].astype(str) + ' ' + df_final['SUB_EVENT3'].astype(str)
df_final['REMARKS'] = (
    df_final['SUB_EVENT1'].fillna('') + ' ' +
    df_final['REMARKS'].fillna('') + ' ' +
    df_final['ResultRecords'].fillna('') + ' ' +
    df_final['SUB_EVENT2'].fillna('') + ' ' +
    df_final['SUB_EVENT3'].fillna('')
).str.strip()


mask = df_final['SUB_EVENT1'].str.contains(r'Para|para', case=False, na=False)
df_final.loc[mask, 'SUB_EVENT'] = 'Para'

mask = df_final['StartStatus'].str.contains(r'DNF', na=True)
df_final.loc[mask, 'RESULT'] = 'DNF'
mask = df_final['StartStatus'].str.contains(r'DNS', na=True)
df_final.loc[mask, 'RESULT'] = 'DNS'
mask = df_final['StartStatus'].str.contains(r'DQ', na=True)
df_final.loc[mask, 'RESULT'] = 'DQ'
mask = df_final['StartStatus'].str.contains(r'NM', na=True)
df_final.loc[mask, 'RESULT'] = 'NM'



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_23004/883706618.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['EVENT_CLASS'] = df_final['EVENT_CLASS1'] + df_final['EVENT_CLASS2']
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_23004/883706618.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['REMARKS'] = (
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_23004/883706618.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [1110]:
df_final.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4320 entries, 0 to 4319
Data columns (total 74 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   MeetingId                4320 non-null   object
 1   COMPETITION              4320 non-null   object
 2   MeetingDate              4320 non-null   object
 3   VENUE                    4320 non-null   object
 4   Organiser                4320 non-null   object
 5   HOST_CITY                4320 non-null   object
 6   Country                  4320 non-null   object
 7   Season                   4320 non-null   object
 8   ATHLETE_ID               4320 non-null   object
 9   StartListId              4320 non-null   object
 10  SUB_EVENT1               814 non-null    object
 11  NAME2                    1720 non-null   object
 12  NationalRelayTeam        238 non-null    object
 13  NAME1                    4315 non-null   object
 14  FIRST_NAME               4082 non-null  

In [1111]:
df_filtered = df_final[['COMPETITION', 'VENUE', 'HOST_CITY', 'ATHLETE_ID', 'SUB_EVENT1', 'NAME2', 'NAME1', 'OTHER_NAME', 'FIRST_NAME', 'LAST_NAME',
                        'SUB_EVENT3', 'SUB_EVENT2', 'REMARKS', 'GENDER', 'RelayId', 'DOB', 'NATIONALITY', 'DATE', 'EVENT', 'EVENT_CLASS1', 'EVENT_CLASS2', 'GROUP',
                        'STAGE', 'TEAM', 'TAG_ID', 'SEED', 'LANE', 'RANK', 'QUALIFICATION', 'RESULT', 'NET_TIME', 'RX_TIME', 'SPLIT_TIME', 'LAP_TIME',
                        'HEAT', 'DICT_RESULTS', 'EVENT_CLASS', 'SUB_EVENT']].copy()


In [1112]:
df_filtered

,COMPETITION,VENUE,HOST_CITY,ATHLETE_ID,SUB_EVENT1,NAME2,NAME1,OTHER_NAME,FIRST_NAME,LAST_NAME,...,QUALIFICATION,RESULT,NET_TIME,RX_TIME,SPLIT_TIME,LAP_TIME,HEAT,DICT_RESULTS,EVENT_CLASS,SUB_EVENT
0,Pesta Sukan Athletics 2025,Home of Athletics,Singapore,2451719,NaN,NaN,Jalea Yeo,NaN,Jalea,Yeo,...,q,13.656,NaN,NaN,NaN,NaN,6,"{1: {'RESULT': '13.656', 'WIND': '-1.4'}}",NaN,NaN
1,Pesta Sukan Athletics 2025,Home of Athletics,Singapore,2451722,NaN,NaN,Leia Ong,NaN,Leia,Ong,...,q,13.716,NaN,NaN,NaN,NaN,1,"{1: {'RESULT': '13.716', 'WIND': '-1.4'}}",NaN,NaN
2,Pesta Sukan Athletics 2025,Home of Athletics,Singapore,2451714,NaN,NaN,Zi Xun Lui,NaN,Zi Xun,Lui,...,q,13.960,NaN,NaN,NaN,NaN,5,"{1: {'RESULT': '13.960', 'WIND': '-1.3'}}",NaN,NaN
3,Pesta Sukan Athletics 2025,Home of Athletics,Singapore,2451705,NaN,NaN,Kristy Kau,NaN,Kristy,Kau,...,q,14.119,NaN,NaN,NaN,NaN,4,"{1: {'RESULT': '14.119', 'WIND': '-1.8'}}",NaN,NaN
4,Pesta Sukan Athletics 2025,Home of Athletics,Singapore,2451717,NaN,NaN,Riana Kaul,NaN,Riana,Kaul,...,q,14.132,NaN,NaN,NaN,NaN,5,"{1: {'RESULT': '14.132', 'WIND': '-1.3'}}",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,Pesta Sukan Athletics 2025,Home of Athletics,Singapore,2452695,Parent Child Relay,Team,"[Sentha Silva, Milan Ocean Ganesh]",NaN,Sentha,Silva,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN
4316,Pesta Sukan Athletics 2025,Home of Athletics,Singapore,2452696,Parent Child Relay,Team,"[Sentha Silva, Milan Ocean Ganesh]",NaN,Milan Ocean,Ganesh,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN
4317,Pesta Sukan Athletics 2025,Home of Athletics,Singapore,2452648,Parent Child Relay,Chun Bao Bao,"[Yao Hui Chun, Shi Han Chun]",NaN,NaN,NaN,...,NaN,DNS,NaN,NaN,NaN,NaN,6,{},NaN,NaN
4318,Pesta Sukan Athletics 2025,Home of Athletics,Singapore,2452649,Parent Child Relay,Chun Bao Bao,"[Yao Hui Chun, Shi Han Chun]",NaN,Yao Hui,Chun,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN


In [1113]:
# Remove extra rows for relay teams

df_filtered = df_filtered[df_filtered['RelayId'].isnull()]

In [1114]:
# TO DO

# Concatenate TEAM + NAME2 and map into TEAM for relay
# Map NAME1 into NAME for everything
# DELETE SUB_EVENT1
# DELETE SUB_EVENT2
# DELETE SUB_EVENT3
# DELETE RelayId
# DELETE EVENT_CLASS1
# DELETE EVENT_CLASS2

df_filtered['TEAM_MAP'] = (df_filtered['TEAM'].fillna('') + ' - ' + df_filtered['NAME2'].fillna('')).str.strip()

df_filtered['NAME'] = df_filtered['NAME1']

df_filtered = df_filtered.drop(columns=['SUB_EVENT1', 'SUB_EVENT2', 'SUB_EVENT3', 'RelayId', 'EVENT_CLASS1', 'EVENT_CLASS2', 'TEAM', 'NAME1', 'NAME2'])

df_filtered.rename(columns={'TEAM_MAP': 'TEAM'}, inplace=True)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_23004/2116764938.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['TEAM_MAP'] = (df_filtered['TEAM'].fillna('') + ' - ' + df_filtered['NAME2'].fillna('')).str.strip()
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_23004/2116764938.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['NAME'] = df_filtered['NAME1']


In [1115]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Roster/')

df_filtered.to_csv('check.csv', sep=',', encoding='utf-8-sig', index=False)

In [1116]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3565 entries, 0 to 4317
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   COMPETITION    3565 non-null   object
 1   VENUE          3565 non-null   object
 2   HOST_CITY      3565 non-null   object
 3   ATHLETE_ID     3565 non-null   object
 4   OTHER_NAME     493 non-null    object
 5   FIRST_NAME     3327 non-null   object
 6   LAST_NAME      3327 non-null   object
 7   REMARKS        3565 non-null   object
 8   GENDER         3565 non-null   object
 9   DOB            3325 non-null   object
 10  NATIONALITY    3565 non-null   object
 11  DATE           3565 non-null   object
 12  EVENT          3565 non-null   object
 13  GROUP          3565 non-null   object
 14  STAGE          3565 non-null   object
 15  TAG_ID         3563 non-null   object
 16  SEED           176 non-null    object
 17  LANE           3521 non-null   object
 18  RANK           2687 non-null   ob

In [1117]:
import datetime

# Set date time format

df_filtered['DATE'] = pd.to_datetime(df_filtered['DATE'], format='mixed', dayfirst=False, utc=False)

df_filtered['DATE'] = df_filtered['DATE'].dt.tz_localize('UTC')

df_filtered['DOB'] = pd.to_datetime(df_filtered['DOB'], format='mixed', dayfirst=True, utc=False)



In [1118]:
df_filtered['DATE']

0      2025-07-26 13:30:00+00:00
1      2025-07-26 13:30:00+00:00
2      2025-07-26 13:30:00+00:00
3      2025-07-26 13:30:00+00:00
4      2025-07-26 13:30:00+00:00
                  ...           
4307   2025-08-03 16:40:00+00:00
4310   2025-08-03 16:40:00+00:00
4313   2025-08-03 16:40:00+00:00
4314   2025-08-03 16:40:00+00:00
4317   2025-08-03 16:40:00+00:00
Name: DATE, Length: 3565, dtype: datetime64[ns, UTC]

In [1119]:
set(df_filtered['EVENT'].to_list())

{'100m',
 '100m Hurdles',
 '110m Hurdles',
 '1500m',
 '1500m Race Walk',
 '200m',
 '200m Hurdles',
 '2x50m MIXED TEAM',
 '400m',
 '400m Hurdles',
 '4x100m',
 '4x100m MIXED TEAM',
 '4x400m',
 '4x400m MIXED TEAM',
 '50m',
 '60m',
 '800m',
 '80m',
 '80m Hurdles',
 'Discus Throw',
 'High Jump',
 'Javelin Throw',
 'Long Jump',
 'Long Jump (Zone)',
 'Pole Vault',
 'Shot Put',
 'Triple Jump',
 'Turbojav Throw'}

In [1120]:
# New schema

# Rename columns. Event Class = 500g for javelin etc (mapped to Event_note). SUB_EVENT=dash under decathlon
# Need to add: Last_name, First_name, Pref_name


#df_filtered.columns = ['LAST_NAME', 'FIRST_NAME', 'OTHER_NAME', 'NATIONALITY', 'GENDER', 'DOB', 'AGE', 'TAG_ID', 'TEAM', 'EVENT', 'SEED', 'STAGE', 'UNIQUE_ID', 'HEAT', 'RESULT', 'WIND', 'RANK', 'LANE', 'REMARKS', 'ATHLETE_ID', 'DIVISION', 'COMPETITION', 'SUB_EVENT', 'EVENT_CLASS', 'DISTANCE', 'POINTS', 'VENUE', 'NAME', 'YEAR', 'DATE']

df_filtered['QUALIFICATION']=''
df_filtered['UNIQUE_ID']=''
df_filtered['REGION']='Local'
df_filtered['CATEGORY_EVENT']=''
df_filtered['SOURCE']='ROSTER'
df_filtered['TIMESTAMP']=''
df_filtered['SESSION']=''
df_filtered['POINTS']=''
df_filtered['DIVISION']=''
df_filtered['AGE']=''
df_filtered['DISTANCE']=''

df_filtered['YEAR']=df_filtered['DATE'].dt.year



# Map miscellaneous fields to final formats                            


df_filtered.loc[df_filtered['EVENT'].str.contains(r'1500m Race Walk', na=True), 'EVENT'] = '1500m Racewalk'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'^4x100m$', na=True), 'EVENT'] = '4 x 100m'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'^4x400m$', na=True), 'EVENT'] = '4 x 400m'
df_filtered.loc[df_filtered['STAGE'].str.contains(r'^Final$', na=True), 'STAGE'] = 'Finals'
df_filtered.loc[df_filtered['STAGE'].str.contains(r'^Heat$', na=True), 'STAGE'] = 'Heats'


# Map general event category 
df_filtered.loc[df_filtered['EVENT'].str.contains(r'^50m$', na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'^60m$', na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'^80m$', na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'^100m$', na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'^200m$', na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'^400m$', na=True), 'CATEGORY_EVENT'] = 'Sprint'

df_filtered.loc[df_filtered['EVENT'].str.contains(r'^800m$', na=True), 'CATEGORY_EVENT'] = 'Mid'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'^1500m$', na=True), 'CATEGORY_EVENT'] = 'Mid'

df_filtered.loc[df_filtered['EVENT'].str.contains(r'^80m Hurdles$', na=True), 'CATEGORY_EVENT'] = 'Hurdles'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'4 x|2x|4x', na=True), 'CATEGORY_EVENT'] = 'Relay'

# Override values
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Discus', na=True), 'CATEGORY_EVENT'] = 'Throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Javelin', na=True), 'CATEGORY_EVENT'] = 'Throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Turbojav', na=True), 'CATEGORY_EVENT'] = 'Throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Shot', na=True), 'CATEGORY_EVENT'] = 'Throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Vault', na=True), 'CATEGORY_EVENT'] = 'Jump'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Throw', na=True), 'CATEGORY_EVENT'] = 'Throw'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Jump', na=True), 'CATEGORY_EVENT'] = 'Jump'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Relay', na=True), 'CATEGORY_EVENT'] = 'Relay'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Steeple', na=True), 'CATEGORY_EVENT'] = 'Steeple'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Walk|Racewalk', na=True), 'CATEGORY_EVENT'] = 'Walk'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Pentathlon', na=True), 'CATEGORY_EVENT'] = 'Pentathlon'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Triathlon', na=True), 'CATEGORY_EVENT'] = 'Triathlon'
df_filtered.loc[df_filtered['EVENT'].str.contains(r'Decathlon', na=True), 'CATEGORY_EVENT'] = 'Decathlon'


# Rename conventions e.g. 100 Meters becomes 100m to be consistent with rest of dataset  

#df_filtered.loc[df_filtered['EVENT'].str.contains(r'100.0 Dash', na=True), 'EVENT'] = '100m'

    

df_filtered = df_filtered.reindex(columns= ['FIRST_NAME', 'LAST_NAME', 'OTHER_NAME', 'NAME', 'RANK', 'TAG_ID', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION',
                                        'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'NATIONALITY',
                                        'DICT_RESULTS', 'YEAR', 'DATE', 'COMPETITION', 'REGION', 'DOB', 'GROUP', 'CATEGORY_EVENT', 'ATHLETE_ID',
                                        'SOURCE', 'REMARKS', 'TIMESTAMP', 'VENUE', 'SUB_EVENT', 'SESSION', 'EVENT_CLASS', 'DISTANCE', 'HOST_CITY', 'RX_TIME', 
                                        'LAP_TIME', 'SPLIT_TIME', 'NET_TIME'])




In [1121]:
df_filtered

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME,LAP_TIME,SPLIT_TIME,NET_TIME
0,Jalea,Yeo,NaN,Jalea Yeo,1,809,Cedar Girls' Secondary School -,NaN,13.656,,...,Home of Athletics,NaN,,NaN,,Singapore,NaN,NaN,NaN,NaN
1,Leia,Ong,NaN,Leia Ong,2,1764,Team Cicada Trackers -,NaN,13.716,,...,Home of Athletics,NaN,,NaN,,Singapore,NaN,NaN,NaN,NaN
2,Zi Xun,Lui,NaN,Zi Xun Lui,3,1223,Nanyang Girls High School -,NaN,13.960,,...,Home of Athletics,NaN,,NaN,,Singapore,NaN,NaN,NaN,NaN
3,Kristy,Kau,NaN,Kristy Kau,4,1760,Team Cicada Trackers -,NaN,14.119,,...,Home of Athletics,NaN,,NaN,,Singapore,NaN,NaN,NaN,NaN
4,Riana,Kaul,NaN,Riana Kaul,5,1772,Team Cicada Trackers -,NaN,14.132,,...,Home of Athletics,NaN,,NaN,,Singapore,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307,NaN,NaN,NaN,"[Nakkeran Thirunavakkarasu, Hrudhaya Nakkeran]",NaN,599,- Runner,NaN,DNS,,...,Home of Athletics,NaN,,NaN,,Singapore,NaN,NaN,NaN,NaN
4310,NaN,NaN,NaN,"[Ethan Goh, Wei Kiat Goh]",NaN,608,- Wei Kiat and Ethan,NaN,DNS,,...,Home of Athletics,NaN,,NaN,,Singapore,NaN,NaN,NaN,NaN
4313,NaN,NaN,NaN,NaN,NaN,606,- Team Bolt,NaN,DNS,,...,Home of Athletics,NaN,,NaN,,Singapore,NaN,NaN,NaN,NaN
4314,NaN,NaN,NaN,"[Sentha Silva, Milan Ocean Ganesh]",NaN,605,- Team,NaN,DNS,,...,Home of Athletics,NaN,,NaN,,Singapore,NaN,NaN,NaN,NaN


In [1122]:
#df_filtered['DOB']=df_filtered.DOB.str.rstrip('00:00:00.000')


In [1123]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Roster/')

df_filtered.to_csv('check.csv', sep=',', encoding='utf-8-sig', index=False)

In [1124]:
df_test = df_filtered.astype(str)

In [1125]:
df_test['DATE']

0       2025-07-26 13:30:00+00:00
1       2025-07-26 13:30:00+00:00
2       2025-07-26 13:30:00+00:00
3       2025-07-26 13:30:00+00:00
4       2025-07-26 13:30:00+00:00
                  ...            
4307    2025-08-03 16:40:00+00:00
4310    2025-08-03 16:40:00+00:00
4313    2025-08-03 16:40:00+00:00
4314    2025-08-03 16:40:00+00:00
4317    2025-08-03 16:40:00+00:00
Name: DATE, Length: 3565, dtype: object

In [1126]:
# Set timestamp

import datetime

timezone = pytz.timezone('UTC')
dt_ms = datetime.datetime.now().replace(tzinfo=timezone)
df_test.loc[:,'TIMESTAMP'] = dt_ms.replace(second=0, microsecond=0)


In [1127]:
df_test['TIMESTAMP']

0       2025-08-15 14:46:00+00:00
1       2025-08-15 14:46:00+00:00
2       2025-08-15 14:46:00+00:00
3       2025-08-15 14:46:00+00:00
4       2025-08-15 14:46:00+00:00
                  ...            
4307    2025-08-15 14:46:00+00:00
4310    2025-08-15 14:46:00+00:00
4313    2025-08-15 14:46:00+00:00
4314    2025-08-15 14:46:00+00:00
4317    2025-08-15 14:46:00+00:00
Name: TIMESTAMP, Length: 3565, dtype: object

In [1128]:
# Load BQ Modified Schema (TEST)

from google.cloud.bigquery import SchemaField

modified_schema = [
            bigquery.SchemaField("FIRST_NAME", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("LAST_NAME", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("OTHER_NAME", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("NAME", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("RANK", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("TAG_ID", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("TEAM", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SEED", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("RESULT", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("QUALIFICATION", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("HEAT", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("LANE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("WIND", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("EVENT", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("DIVISION", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("STAGE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("POINTS", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("AGE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("GENDER", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("UNIQUE_ID", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("NATIONALITY", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("DICT_RESULTS", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("YEAR", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("DATE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("COMPETITION", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("REGION", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("DOB", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("GROUP", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("CATEGORY_EVENT", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("ATHLETE_ID", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SOURCE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("REMARKS", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("TIMESTAMP", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("VENUE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SUB_EVENT", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SESSION", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("EVENT_CLASS", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("DISTANCE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("HOST_CITY", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("RX_TIME", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("LAP_TIME", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SPLIT_TIME", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("NET_TIME", "STRING", mode="NULLABLE"),
]

In [1133]:
# Load BQ Modified Schema (PRODUCTION)

from google.cloud.bigquery import SchemaField

df_prod = df_test.iloc[:,:-3]


modified_schema = [
            bigquery.SchemaField("FIRST_NAME", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("LAST_NAME", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("OTHER_NAME", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("NAME", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("RANK", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("TAG_ID", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("TEAM", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SEED", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("RESULT", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("QUALIFICATION", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("HEAT", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("LANE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("WIND", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("EVENT", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("DIVISION", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("STAGE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("POINTS", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("AGE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("GENDER", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("UNIQUE_ID", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("NATIONALITY", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("DICT_RESULTS", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("YEAR", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("DATE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("COMPETITION", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("REGION", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("DOB", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("GROUP", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("CATEGORY_EVENT", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("ATHLETE_ID", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SOURCE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("REMARKS", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("TIMESTAMP", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("VENUE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SUB_EVENT", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SESSION", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("EVENT_CLASS", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("DISTANCE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("HOST_CITY", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("RX_TIME", "STRING", mode="NULLABLE"),
           
]

In [1134]:
df_prod

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,Jalea,Yeo,nan,Jalea Yeo,1,809,Cedar Girls' Secondary School -,nan,13.656,,...,ROSTER,SB,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
1,Leia,Ong,nan,Leia Ong,2,1764,Team Cicada Trackers -,nan,13.716,,...,ROSTER,,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
2,Zi Xun,Lui,nan,Zi Xun Lui,3,1223,Nanyang Girls High School -,nan,13.960,,...,ROSTER,PB,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
3,Kristy,Kau,nan,Kristy Kau,4,1760,Team Cicada Trackers -,nan,14.119,,...,ROSTER,PB,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
4,Riana,Kaul,nan,Riana Kaul,5,1772,Team Cicada Trackers -,nan,14.132,,...,ROSTER,PB,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307,nan,nan,nan,"['Nakkeran Thirunavakkarasu', 'Hrudhaya Nakker...",nan,599,- Runner,nan,DNS,,...,ROSTER,Parent Child Relay,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
4310,nan,nan,nan,"['Ethan Goh', 'Wei Kiat Goh']",nan,608,- Wei Kiat and Ethan,nan,DNS,,...,ROSTER,Parent Child Relay,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
4313,nan,nan,nan,nan,nan,606,- Team Bolt,nan,DNS,,...,ROSTER,Parent Child Relay,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
4314,nan,nan,nan,"['Sentha Silva', 'Milan Ocean Ganesh']",nan,605,- Team,nan,DNS,,...,ROSTER,Parent Child Relay,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan


In [1019]:
gender_pattern = 'Male|Female|Mixed'
year_pattern = '2025'
format_string = "%Y-%m-%d %H:%M:%S%z"

#df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
df['YEAR'] = df['YEAR'].astype(str)
df['DATE'] = df['DATE'].astype(str)
df['DOB'] = df['DOB'].astype(str)
#df['event_date_dt'] = pd.to_datetime(df['event_date'], format='mixed', dayfirst=False, utc=True)

stage_list = ['Finals', 'Final', 'Semifinal', 'Heats', 'Qualification', '', ' ']

event_list = ['100m', '400m', '800m', '1500m', '1500m Walk', '3000m', '5000m', '10,000m', '80m Hurdles', '110m Hurdles', '100m Hurdles', '200m Hurdles', '400m Hurdles', 
              'High Jump', 'Discus', 'Discus Throw', 'Triple Jump', 'Pole Vault','Long Jump', 'Javelin', 'Javelin Throw', 'Shot Put', 'High Jump', '2000m SC', '3000m Walk', 
              '5000m Walk', 'Hammer Throw', '200m', '3000m SC', '4 x 100m Relay', '4 x 400m Relay'] 
category_list = ['Throw', 'Jump', 'Relay', 'Long', 'Mid', 'Sprint', 'Road', 'Cross Country', 'Steeple', 'Hurdles', 'Walk', 'Marathon', 'Triathlon', 'Decathlon', 'Pentathlon']

for index, row in df.iterrows():
        
    assert re.search(gender_pattern, row['GENDER']) is not None   # check format of gender input
    assert row['NAME'] != None
   # assert len(row['NAME'])>5
    assert row['DATE'] != None
    assert row['EVENT'] != None 
    assert row['COMPETITION'] != None 
    assert (row['REGION'] == 'Local' or row['REGION'] == 'International')
    assert row['DATE'] is not None
    assert row['CATEGORY_EVENT'] in category_list
    assert len(row['DATE'])>3
    assert re.search('\)|\(', row['EVENT']) is None  # make sure there are no brackets in the event column
    assert re.search(year_pattern, row['YEAR']) is not None   # check format of year
    assert row['STAGE'] in stage_list 
    assert row['EVENT'] in event_list 
    assert datetime.datetime.strptime(row['DATE'], format_string)
    assert re.search('Para|para', row['EVENT']) is None
    

    
 #   assert not row['event_date_dt'] is pd.NaT
    
    
    # Check column locations before loading

    assert len(df.columns) == 43
    assert df.columns.get_loc('RESULT') == 8
    assert df.columns.get_loc('AGE') == 17
    assert df.columns.get_loc('WIND') == 12
    assert df.columns.get_loc('COMPETITION')==24
    
   

 

print('PASSED')

df

AssertionError: 

In [1023]:
# APPEND to new table into BigQuery with modified schema (TEST

client= bigquery.Client()

job_config = bigquery.LoadJobConfig(
    schema=modified_schema,
    write_disposition="WRITE_APPEND",  # or WRITE_APPEND, WRITE_EMPTY, WRITE_TRUNCATE#
)

job = client.load_table_from_dataframe(df_test, 'saa-analytics.results.TEST', job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=saa-analytics, location=US, id=fbdf58b6-189a-40fd-ba0f-841d8bd352c6>

In [1025]:
# Extract to check that loading was successful

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT * FROM `saa-analytics.results.TEST`
WHERE COMPETITION='Pesta Sukan Athletics 2025'

"""

new_table = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)


Downloading: 100%|█████████████████████████████████████████████████████████████|


In [1026]:
new_table

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME,LAP_TIME,SPLIT_TIME,NET_TIME
0,Jalea,Yeo,nan,Jalea Yeo,1,809,Cedar Girls' Secondary School -,nan,13.656,,...,Home of Athletics,nan,,nan,,Singapore,nan,nan,nan,nan
1,Leia,Ong,nan,Leia Ong,2,1764,Team Cicada Trackers -,nan,13.716,,...,Home of Athletics,nan,,nan,,Singapore,nan,nan,nan,nan
2,Zi Xun,Lui,nan,Zi Xun Lui,3,1223,Nanyang Girls High School -,nan,13.960,,...,Home of Athletics,nan,,nan,,Singapore,nan,nan,nan,nan
3,Kristy,Kau,nan,Kristy Kau,4,1760,Team Cicada Trackers -,nan,14.119,,...,Home of Athletics,nan,,nan,,Singapore,nan,nan,nan,nan
4,Riana,Kaul,nan,Riana Kaul,5,1772,Team Cicada Trackers -,nan,14.132,,...,Home of Athletics,nan,,nan,,Singapore,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3560,nan,nan,nan,"['Nakkeran Thirunavakkarasu', 'Hrudhaya Nakker...",nan,599,- Runner,nan,DNS,,...,Home of Athletics,nan,,nan,,Singapore,nan,nan,nan,nan
3561,nan,nan,nan,"['Ethan Goh', 'Wei Kiat Goh']",nan,608,- Wei Kiat and Ethan,nan,DNS,,...,Home of Athletics,nan,,nan,,Singapore,nan,nan,nan,nan
3562,nan,nan,nan,nan,nan,606,- Team Bolt,nan,DNS,,...,Home of Athletics,nan,,nan,,Singapore,nan,nan,nan,nan
3563,nan,nan,nan,"['Sentha Silva', 'Milan Ocean Ganesh']",nan,605,- Team,nan,DNS,,...,Home of Athletics,nan,,nan,,Singapore,nan,nan,nan,nan


In [1136]:
df_prod = df_prod.astype(str)

In [1137]:
df_prod

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,Jalea,Yeo,nan,Jalea Yeo,1,809,Cedar Girls' Secondary School -,nan,13.656,,...,ROSTER,SB,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
1,Leia,Ong,nan,Leia Ong,2,1764,Team Cicada Trackers -,nan,13.716,,...,ROSTER,,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
2,Zi Xun,Lui,nan,Zi Xun Lui,3,1223,Nanyang Girls High School -,nan,13.960,,...,ROSTER,PB,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
3,Kristy,Kau,nan,Kristy Kau,4,1760,Team Cicada Trackers -,nan,14.119,,...,ROSTER,PB,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
4,Riana,Kaul,nan,Riana Kaul,5,1772,Team Cicada Trackers -,nan,14.132,,...,ROSTER,PB,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307,nan,nan,nan,"['Nakkeran Thirunavakkarasu', 'Hrudhaya Nakker...",nan,599,- Runner,nan,DNS,,...,ROSTER,Parent Child Relay,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
4310,nan,nan,nan,"['Ethan Goh', 'Wei Kiat Goh']",nan,608,- Wei Kiat and Ethan,nan,DNS,,...,ROSTER,Parent Child Relay,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
4313,nan,nan,nan,nan,nan,606,- Team Bolt,nan,DNS,,...,ROSTER,Parent Child Relay,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
4314,nan,nan,nan,"['Sentha Silva', 'Milan Ocean Ganesh']",nan,605,- Team,nan,DNS,,...,ROSTER,Parent Child Relay,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan


In [1138]:
# APPEND to new table into BigQuery with modified schema (PRODUCTION)

client= bigquery.Client()

job_config = bigquery.LoadJobConfig(
    schema=modified_schema,
    write_disposition="WRITE_APPEND",  # or WRITE_APPEND, WRITE_EMPTY, WRITE_TRUNCATE#
)

job = client.load_table_from_dataframe(df_prod, 'saa-analytics.results.PRODUCTION', job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=saa-analytics, location=US, id=66b74e3c-0709-440d-838f-5dfab83f5331>

In [1139]:
# Extract to check that loading was successful

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT * FROM `saa-analytics.results.PRODUCTION`
WHERE COMPETITION='Pesta Sukan Athletics 2025'

"""

new_table = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)


Downloading: 100%|█████████████████████████████████████████████████████████████|


In [1140]:
new_table

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,Jalea,Yeo,nan,Jalea Yeo,1,809,Cedar Girls' Secondary School -,nan,13.656,,...,ROSTER,SB,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
1,Leia,Ong,nan,Leia Ong,2,1764,Team Cicada Trackers -,nan,13.716,,...,ROSTER,,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
2,Zi Xun,Lui,nan,Zi Xun Lui,3,1223,Nanyang Girls High School -,nan,13.960,,...,ROSTER,PB,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
3,Kristy,Kau,nan,Kristy Kau,4,1760,Team Cicada Trackers -,nan,14.119,,...,ROSTER,PB,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
4,Riana,Kaul,nan,Riana Kaul,5,1772,Team Cicada Trackers -,nan,14.132,,...,ROSTER,PB,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3560,nan,nan,nan,"['Nakkeran Thirunavakkarasu', 'Hrudhaya Nakker...",nan,599,- Runner,nan,DNS,,...,ROSTER,Parent Child Relay,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
3561,nan,nan,nan,"['Ethan Goh', 'Wei Kiat Goh']",nan,608,- Wei Kiat and Ethan,nan,DNS,,...,ROSTER,Parent Child Relay,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
3562,nan,nan,nan,nan,nan,606,- Team Bolt,nan,DNS,,...,ROSTER,Parent Child Relay,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
3563,nan,nan,nan,"['Sentha Silva', 'Milan Ocean Ganesh']",nan,605,- Team,nan,DNS,,...,ROSTER,Parent Child Relay,2025-08-15 14:46:00+00:00,Home of Athletics,nan,,nan,,Singapore,nan
